In [1]:
import json
from utils import *
from plan_utils import *

with open('/home/wuy/DB/pg_mem_data/tpch/tiny_plans.json') as f:
    plans = json.load(f)

plan = plans[0]
plan['database_id']=0
plan['plan_id']=0

statistics_file = '/home/wuy/DB/DACE/data/workload1/statistics.json'
feature_statistics = load_json(statistics_file)
add_numerical_scalers(feature_statistics)
op_name_to_one_hot = get_op_name_to_one_hot(feature_statistics)

configs = {"pad_length":20, "max_runtime":30000, "node_length":18, "loss_weight":0.5, "hidden_dim": 128,
           "mlp_activation": "ReLU", "transformer_activation": "gelu", "mlp_dropout": 0.3, "transformer_dropout": 0.2}
plan_meta = get_plan_encoding(plan, configs, op_name_to_one_hot, plan_parameters, feature_statistics)
# print(plan_meta)  # seq_encoding, run_times, attention_mask, loss_mask, database_id

/home/wuy/software/anaconda3/envs/dace/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from torch.utils.data import DataLoader
seq, run_time, attention_mask, loss_mask = plan_meta[:-1]
dataset = DACEDataset([seq], [attention_mask], [loss_mask], [run_time]) # return seq, attn_mask, loss_mask, run_time
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

In [3]:
from model import DACELora, PL_DACE
model = DACELora(
    configs["node_length"],
    configs["hidden_dim"],
    1,
    configs["mlp_activation"],
    configs["transformer_activation"],
    configs["mlp_dropout"],
    configs["transformer_dropout"],
)



In [22]:
# Define DACE Loss
def DACE_loss(est_run_times, run_times, loss_mask, loss_fn):
    # est_run_times: (batch, seq_len)
    # run_times: (batch, seq_len)
    # loss_mask: (batch, seq_len)
    # return: loss (batch,)
    # don't calculate the loss of padding nodes, set them to 0
    # loss = torch.max(est_run_times / run_times, run_times / est_run_times)
    loss = loss_fn(est_run_times[0], run_times[0])
    print(f"loss {loss}")
    
    return loss

In [23]:
import torch.nn as nn
model.train()
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
for data in dataloader:
    seqs, attn_masks, loss_masks, run_times = data
    est_run_times = model(seqs, attn_masks)
    loss = DACE_loss(est_run_times, run_times, loss_masks, loss_fn)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()    

loss 0.20530235767364502


In [14]:
est_run_times.shape

torch.Size([1, 20])